In [23]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view #for easy minima without using for loops
from math import *
from collections import deque

Wisniowski and Rickman (2013): Fast MOID calculation

because Gronchi's code is too complicated


In [120]:
# Step 1: ~(cover yourself in oil)~ rotate matrix
# Break out the 403 notes: Compute dcm_a, dcm_b, convert from a to b to get dcm_b_a 
# (dcm of b wrt A, where dcm_b is an orientation and a is a )
# Passive rotation

#D_HF = D_HG @ D_GF
#D_BA = D_B0 @ D_0A = D_B0 @ D_A0.T   from A to B
#D_AB = D_A0 @ D_0B
def rotate(inc_a,argpe_a,omega_a,inc_b,argpe_b,omega_b):
    """
    Remember all angles are in radians and negative!!!
    """
    dcm_a = np.array([[ np.cos(omega_a), np.sin(omega_a), 0],
                      [-np.sin(omega_a), np.cos(omega_a), 0],
                      [               0,               0, 1]]) @ np.array([[ 1, 0, 0],
                      [0,  np.cos(inc_a), np.sin(inc_a)],
                      [0, -np.sin(inc_a), np.cos(inc_a)]]) @ np.array([[ np.cos(argpe_a),np.sin(argpe_a), 0],
                      [-np.sin(argpe_a), np.cos(argpe_a), 0],
                      [                         0,                        0, 1]])
    dcm_b = np.array([[ np.cos(omega_b), np.sin(omega_b), 0],
                      [-np.sin(omega_b), np.cos(omega_b), 0],
                      [               0,               0, 1]]) @ np.array([[ np.cos(inc_b), 0, -np.sin(inc_b)],
                      [             0, 1,             0],
                      [ np.sin(inc_a), 0, np.cos(inc_b)]]) @ np.array([[ np.cos(argpe_b),np.sin(argpe_b), 0],
                      [-np.sin(argpe_a), np.cos(argpe_a), 0],
                      [                         0,                        0, 1]])
    dcm_ba = dcm_b@dcm_a.T #dcm from A to B
    # Now we convert this back into Euler using 403 formula
    # in the 403 notes, phi = argpe, theta = inc, psi = omega
    # remember you need to make this negative
    inc_ba = -np.arccos(dcm_ba[2][2])
    argpe_ba = -np.arctan2(dcm_ba[2][0], -dcm_ba[2][1])
    omega_ba = -np.arctan2(dcm_ba[0][2], dcm_ba[1][2]) #I sure hope this works.
    return inc_ba, argpe_ba, omega_ba

# to find local minima: D_min_i should be smaller than both preceding and following values
# What this means: Get log of all distances D_squared, run function to detect minima (I'll code this myself)
def minima(D_sq):
    """
    Strategy:
    1) Append 0th element to end of D_sq (for wraparound purposes)
    2) Sliding window view to create giant array of n x 3
    3) Find minima
    4) Knowing center column is either [1: (n+1)] in np.append or D_sq itself in np.pad,
    subtract D_sq from that and find zeros. Indices that are 0 are local minima.
    5) Return indices for easy math
    """

    D_sq_wrap = np.pad(D_sq,(1,1),'wrap')
    q = sliding_window_view(D_sq_wrap,3) # Produces n-1 size array, slice first element
    # print(np.min(q,axis=1)-D_sq)
    return np.argwhere((np.min(q,axis=1)-D_sq) == 0 ).T[0] #Returns indices, needed to find L and nu
    # return D_sq[np.argwhere((np.min(q,axis=1)-D_sq) == 0 ).T[0]] #Returns actual values if you so please

# def cartesian(L, nu, a_a,e_a, a_b,e_b,i_b,argpe_b,raan_b):
#     r_a = (a_a*(1-e_a**2)) / (1+e_a*cos(L))
#     x_a = r_a*cos(L)
#     y_a = r_a*sin(L)

#     r_b = (a_b*(1-e_b**2)) / (1+e_b*cos(nu))
#     x_b = r_b*( cos(raan_b)*cos(argpe_b + nu) - sin(raan_b)*sin(argpe_b + nu)*cos(i_b) )
#     y_b = r_b*( sin(raan_b)*cos(argpe_b + nu) + cos(raan_b)*sin(argpe_b + nu)*cos(i_b) )
#     z_b = r_b*sin(argpe_b + nu)*sin(i_b)
#     return np.array([x_a,y_a,0]), np.array([x_b, y_b, z_b])

def cartesian_a(L, a_a,e_a):
    r_a = (a_a*(1-e_a**2)) / (1+e_a*cos(L))
    x_a = r_a*cos(L)
    y_a = r_a*sin(L)
    return np.array([x_a,y_a,0])

def cartesian_b(nu, a_b,e_b,i_b,argpe_b,raan_b):
    r_b = (a_b*(1-e_b**2)) / (1+e_b*cos(nu))
    x_b = r_b*( cos(raan_b)*cos(argpe_b + nu) - sin(raan_b)*sin(argpe_b + nu)*cos(i_b) )
    y_b = r_b*( sin(raan_b)*cos(argpe_b + nu) + cos(raan_b)*sin(argpe_b + nu)*cos(i_b) )
    z_b = r_b*sin(argpe_b + nu)*sin(i_b)
    return np.array([x_b, y_b, z_b])

def dist(fum,foe):
    """Takes two arrays and calcs the (squared) dist between them"""
    return (fum[0]-foe[0])**2 + (fum[1]-foe[1])**2 + (fum[2]-foe[2])**2

def parallel_target(L, nu, a_a,e_a, a_b,e_b,i_b,argpe_b,raan_b, D_squared_minima, step, refine_factor, targstep):
    A = deque([0,0,0]) #L-1, L, L+1
    B = deque([0,0,0]) #nu-1, nu, nu+1
    A[1] = cartesian_a(L, a_a,e_a)
    B[1] = cartesian_b(nu, a_b,e_b,i_b,argpe_b,raan_b)
    al, ar, bl, br = True, True, True, True # Set to true to trigger creation of all
    moid_parallel = D_squared_minima
    L_para = L
    nu_para = nu
    while step > targstep: #Initial targeting loop
        if al: # Create left point for A
            A[0] = cartesian_a(L-step, a_a,e_a)
        if ar: # Create right point for A
            A[2] = cartesian_a(L+step, a_a,e_a)
        if bl: # Create left point for B
            B[0] = cartesian_b(nu-step, a_b,e_b,i_b,argpe_b,raan_b)
        if br: # Create right point for B
            B[2] = cartesian_b(nu+step, a_b,e_b,i_b,argpe_b,raan_b)
        
        D = [] #first 3: A1L, next 3: A1, next 3: A1R
        for i in range(3): #calculate distances
            for j in range(3): #B iter
                D.append(dist(A[i],B[j]))
        D_min_index = np.argmin(D) #Directly compare as the doctor ordered
        moid_candidate = D[D_min_index]
        print("D:", np.sqrt(np.array(D))/(1.496e8), "Min index:", D_min_index, "Min D:", sqrt(moid_candidate)/(1.496e8), "AU")
        if moid_candidate < moid_parallel: #if smaller, figure out which one is smaller and shift
            print("Old A:",A)
            if D_min_index < 3: #A1L is smaller, shift right
                L -= step
                al = True #Recalculate A1L, old A1L is now A1 and this is technically A1R which is obsolete
                ar = False 
                A.rotate(1) #roll to make A1L the new A1
                print("L shift left", L)
            elif D_min_index < 6: #A1 is smaller, no shift
                al, ar = False, False
                print("L no shift", L)
            else: #A1R is smaller, shift left
                L += step
                al = False
                ar = True
                A.rotate(-1) #roll to make A1R the new A1
                print("L shift right", L)
            print("New A:", A)

            print("Old B:",B)
            if D_min_index % 3 == 0: #B1L is smaller, shift left
                nu -= step
                bl = True
                br = False
                B.rotate(1)
                print("Nu shift left", nu)
            elif D_min_index % 3 == 1: #B1 is smaller, no shift
                bl, br = False, False
                print("Nu no shift", nu)
            else: #B1R is smaller, shift left
                nu += step
                bl = False
                br = True
                B.rotate(-1)
                print("Nu shift right", nu)
            print("New B:", B)
            #record after shift
            moid_parallel = moid_candidate
            L_para = L
            nu_para = nu
            print("Shifted candidate:", sqrt(moid_parallel)/(1.496e8), "AU |Step:", step, "L:", L, "nu:", nu,"Index", D_min_index)
        elif moid_candidate >= moid_parallel: #if equal, converged, break and reduce step
            # no need for that second loop, just run as many times as needed until step is small
            step *= refine_factor
            print("Refine factor applied, step:", step,"Old L:", L, "Old nu:", nu,"New L:", L_para, "New nu:", nu_para)
            # if moid_candidate > moid_parallel: #Reload from backup
            L = L_para
            nu = nu_para
            A[1] = cartesian_a(L, a_a,e_a)
            B[1] = cartesian_b(nu, a_b,e_b,i_b,argpe_b,raan_b)
            al, ar, bl, br = True, True, True, True # Set to true to trigger creation of all
        # else: #this only happens due to floating point error
        #     raise ValueError("Why is candidate larger than initial parallel minima")
        print("Current candidate:", sqrt(moid_parallel)/(1.496e8), "AU |Step:", step, "L:", L, "nu:", nu,"Index", D_min_index)

    # after loop, save moid_parallel and corresponding L and nu
    return moid_parallel, L, nu

def moid(a_a,e_a,i_1,argpe_1,raan_1, a_b,e_b,i_2,argpe_2,raan_2,delta_nu_scan=0.009,delta_nu_refine=0.004,refine_factor=0.15,init_targstep=5e-6,final_targstep=1e-14):
    print([a_a,e_a,i_1,argpe_1,raan_1])
    print([a_b,e_b,i_2,argpe_2,raan_2])
    i_b, argpe_b, raan_b = rotate(-i_1,-argpe_1,-raan_1,-i_2,-argpe_2,-raan_2)
    # Need to record L and nu for each index
    # you calculate L

    # Scanning
    nu_list = np.arange(0,2*pi,delta_nu_scan)
    L_list = []
    D_squared = []
    # for i in range(nu_list.size):
    for nu in nu_list:
        r_b = (a_b*(1-e_b**2)) / (1+e_b*cos(nu))
        x_b = r_b*( cos(raan_b)*cos(argpe_b + nu) - sin(raan_b)*sin(argpe_b + nu)*cos(i_b) )
        y_b = r_b*( sin(raan_b)*cos(argpe_b + nu) + cos(raan_b)*sin(argpe_b + nu)*cos(i_b) )
        z_b = r_b*sin(argpe_b + nu)*sin(i_b)
        # tan L = y_b/x_b, want tangent to apply atan2
        L = atan2(y_b,x_b)
        r_a = (a_a*(1-e_a**2)) / (1+e_a*cos(L))
        rho_b = sqrt(x_b**2 + y_b**2)
        L_list.append(L)
        D_squared.append(z_b**2 + (rho_b -r_a)**2)
    D_squared = np.array(D_squared)
    L_list = np.array(L_list)
    D_min_i = minima(D_squared).astype(int)
    # print(type(D_min_i[0]))
    print("Min D index",D_min_i,"D_min-1", np.sqrt(D_squared[D_min_i-1])/1.496e8,"D_min",np.sqrt(D_squared[D_min_i])/1.496e8, "D_min+1", np.sqrt(D_squared[D_min_i+1])/1.496e8)

    #initial tuning, iterate through all minima. Find smallest and put that into final tuning
    L_sel = L_list[D_min_i]
    nu_sel = nu_list[D_min_i]
    step = delta_nu_refine
    
    L_para_init = []
    nu_para_init = []
    moid_para_sq_init = []
    
    for i in range(L_sel.size): #all minima
        L = L_sel[i]
        nu = nu_sel[i]
        print("Beginning parallel L:",L,"nu:",nu)
        moid_para_sq, L_para, nu_para = parallel_target(L, nu, a_a,e_a, a_b,e_b,i_b,argpe_b,raan_b, D_squared[D_min_i[i]], step, refine_factor, init_targstep)
        L_para_init.append(L_para)
        nu_para_init.append(nu_para)
        moid_para_sq_init.append(moid_para_sq)

    # check which moid is smaller
    moid_final_i = np.argmin(moid_para_sq_init)
    moid_prefinal_sq = moid_para_sq_init[moid_final_i]
    L_final = L_para_init[moid_final_i]
    nu_final = nu_para_init[moid_final_i]
    print("\n Before final:", sqrt(moid_prefinal_sq)/(1.496e8), "AU |L:", L_final, "nu:", nu_final, "Choices:", np.sqrt(moid_para_sq_init)/1.496e8,"\n\n\n\n")

    moid_final_sq, L_final, nu_final = parallel_target(L_final, nu_final, a_a,e_a, a_b,e_b,i_b,argpe_b,raan_b, moid_prefinal_sq, step, refine_factor, final_targstep)

    return nu_list[D_min_i], L_list[D_min_i], np.sqrt(D_squared[D_min_i]), sqrt(moid_final_sq), L_final, nu_final


In [121]:
AU = 1.496e+8 #km to AU
# AU = 149597870.691
#a, e, i, argpe, raan state vectors
o_earth =   [1.4765067E+08, 9.1669995E-03, 4.2422693E-03, 6.64375167E+01, 1.4760836E+01]
o_apophis = [1.3793939E+08, 1.9097084E-01, 3.3356539E+00, 1.2919949E+02, 2.0381969E+02]
o_yr4 =     [3.7680703E+08, 6.6164147E-01, 3.4001497E+00, 1.3429905E+02, 2.7147904E+02]
o_3iatlas = [-3.9552667E+07, 6.1469268E+00, 1.7512507E+02, 1.2817255E+02, 3.2228906E+02]

#convert to radians, oopsies
o_earth[2:] = [x*pi/180 for x in o_earth[2:]]
o_apophis[2:] = [x*pi/180 for x in o_apophis[2:]]
o_yr4[2:] = [x*pi/180 for x in o_yr4[2:]]
o_3iatlas[2:] = [x*pi/180 for x in o_3iatlas[2:]]

#test asteroid
o_21lutetia = [(2.036/(1-0.164))*AU, 0.164, 0, 250.227, 0]
o_27110 = [(1.77550824/(1-0.1928808))*AU, 0.1928808, 3.43901, 216.20834, 140.55651]
o_21lutetia[2:] = [x*pi/180 for x in o_21lutetia[2:]]
o_27110[2:] = [x*pi/180 for x in o_27110[2:]]
# results_test18 = moid(*o_21lutetia, *o_27110)
# print(f"MOID Test 18 = {results_test18[3]:.1f} km = {results_test18[3]/AU:.6f} AU")
# print(f"Initial MOID Test 18 = {results_test18[2]} km = {results_test18[2]/AU} AU")

moid_earth_apophis = moid(*o_earth, *o_apophis)
print(f"MOID Earth-Apophis = {moid_earth_apophis[3]:.1f} km = {moid_earth_apophis[3]/AU:.6f} AU")
print(f"MOID Earth-Apophis L = {np.rad2deg(moid_earth_apophis[4]):.3f} deg | nu = {np.rad2deg(moid_earth_apophis[5]):.3f} deg")
print(f"Initial MOID Earth-Apophis = {moid_earth_apophis[2]} km = {moid_earth_apophis[2]/AU} AU")


[147650670.0, 0.0091669995, 7.404156704127508e-05, 1.1595534132637177, 0.2576251885469097]
[137939390.0, 0.19097084, 0.05821814326198968, 2.2549564923974885, 3.5573246708941615]
Min D index [233 471] D_min-1 [0.00244128 0.04768017] D_min [0.00198476 0.04764961] D_min+1 [0.0028555  0.04767904]
Beginning parallel L: -1.11946734455736 nu: 2.097
D: [0.00201637 0.00440608 0.00817005 0.00440361 0.00198476 0.00451986
 0.00809969 0.00440192 0.00224605] Min index: 4 Min D: 0.001984757542582331 AU
Refine factor applied, step: 0.0006 Old L: -1.11946734455736 Old nu: 2.097 New L: -1.11946734455736 New nu: 2.097
Current candidate: 0.00198475754258232 AU |Step: 0.0006 L: -1.11946734455736 nu: 2.097 Index 4
D: [0.00196968 0.00207116 0.0023278  0.00205485 0.00198476 0.00209161
 0.0022936  0.00206987 0.00200664] Min index: 0 Min D: 0.001969676824871916 AU
Old A: deque([array([ 6.40586415e+07, -1.32365018e+08,  0.00000000e+00]), array([ 6.41377328e+07, -1.32325906e+08,  0.00000000e+00]), array([ 6.42168

In [112]:
moid_earth_yr4 = moid(*o_earth, *o_yr4)
print(f"MOID Earth-2024 YR4 = {moid_earth_yr4[3]:.1f} km = {moid_earth_yr4[3]/AU:.6f} AU")
print(f"MOID Earth-2024 YR4 L = {np.rad2deg(moid_earth_yr4[4]):.3f} deg | nu = {np.rad2deg(moid_earth_yr4[5]):.3f} deg")
print(f"Initial MOID Earth-2024 YR4 = {moid_earth_yr4[2]} km = {moid_earth_yr4[2]/AU} AU")

Min D index [ 98 604] D_min-1 [0.01483065 0.05380904] D_min [0.01423038 0.05377475] D_min+1 [0.01435836 0.05391988]
Beginning parallel L: 2.926587451710546 nu: 0.8819999999999999
D: [0.01441194 0.0147784  0.01628086 0.01495061 0.01423038 0.01474329
 0.01646376 0.01477736 0.01419407] Min index: 8 Min D: 0.014194074648488755 AU
Old A: deque([array([-1.45412944e+08,  3.23653020e+07,  0.00000000e+00]), array([-1.45542401e+08,  3.17836461e+07,  0.00000000e+00]), array([-1.45669510e+08,  3.12014678e+07,  0.00000000e+00])])
L shift right 2.930587451710546
New A: deque([array([-1.45542401e+08,  3.17836461e+07,  0.00000000e+00]), array([-1.45669510e+08,  3.12014678e+07,  0.00000000e+00]), array([-1.45412944e+08,  3.23653020e+07,  0.00000000e+00])])
Old B: deque([array([-1.45348654e+08,  3.23500440e+07,  2.15501354e+06]), array([-1.45686070e+08,  3.18150206e+07,  2.12378003e+06]), array([-1.46022815e+08,  3.12780970e+07,  2.09243260e+06])])
Nu shift right 0.8859999999999999
New B: deque([array([

In [108]:
moid_apophis_yr4 = moid(*o_apophis, *o_yr4)
print(f"MOID Apophis-2024 YR4 = {moid_apophis_yr4[3]:.1f} km = {moid_apophis_yr4[3]/AU:.6f} AU")
print(f"MOID Apophis-2024 YR4 L = {np.rad2deg(moid_apophis_yr4[4]):.3f} deg | nu = {np.rad2deg(moid_apophis_yr4[5]):.3f} deg")
print(f"Initial MOID Apophis-2024 YR4 = {moid_apophis_yr4[2]} km = {moid_apophis_yr4[2]/AU} AU")

Min D index [0] D_min-1 [0.02983539] D_min [0.02983139] D_min+1 [0.02984303]
Beginning parallel L: 1.2619654063634491 nu: 0.0
D: [0.02985066 0.03025993 0.03103683 0.0298015  0.02983139 0.0302444
 0.03014543 0.02979394 0.02982699] Min index: 7 Min D: 0.029793940484962016 AU
Old A: deque([array([3.86326087e+07, 1.19438553e+08, 0.00000000e+00]), array([3.81807733e+07, 1.19674333e+08, 0.00000000e+00]), array([3.77277382e+07, 1.19908626e+08, 0.00000000e+00])])
L shift right 1.2659654063634491
New A: deque([array([3.81807733e+07, 1.19674333e+08, 0.00000000e+00]), array([3.77277382e+07, 1.19908626e+08, 0.00000000e+00]), array([3.86326087e+07, 1.19438553e+08, 0.00000000e+00])])
Old B: deque([array([3.92166638e+07, 1.21247791e+08, 4.04074764e+06]), array([3.87319169e+07, 1.21401845e+08, 4.07778362e+06]), array([3.82467970e+07, 1.21554729e+08, 4.11478034e+06])])
Nu no shift 0.0
New B: deque([array([3.92166638e+07, 1.21247791e+08, 4.04074764e+06]), array([3.87319169e+07, 1.21401845e+08, 4.0777836

In [109]:
moid_earth_3iatlas = moid(*o_earth, *o_3iatlas)
print(f"MOID Earth-3I/ATLAS = {moid_earth_3iatlas[3]:.1f} km = {moid_earth_3iatlas[3]/AU:.6f} AU")
print(f"MOID Earth-3I/ATLAS L = {np.rad2deg(moid_earth_3iatlas[4]):.3f} deg | nu = {np.rad2deg(moid_earth_3iatlas[5]):.3f} deg")
print(f"Initial MOID Earth-3I/ATLAS = {moid_earth_3iatlas[2]} km = {moid_earth_3iatlas[2]/AU} AU")

Min D index [  0 349] D_min-1 [0.37830307 0.90588984] D_min [0.37829982 0.90580939] D_min+1 [0.37832905 0.90591208]
Beginning parallel L: 1.4166941809530742 nu: 0.0
D: [0.37850094 0.3783634  0.37830111 0.37838083 0.37829982 0.37829408
 0.37831734 0.37829288 0.3783437 ] Min index: 7 Min D: 0.37829287523468386 AU
Old A: deque([array([2.32113033e+07, 1.45586774e+08, 0.00000000e+00]), array([2.26295906e+07, 1.45683724e+08, 0.00000000e+00]), array([2.20474742e+07, 1.45778353e+08, 0.00000000e+00])])
L shift right 1.4206941809530742
New A: deque([array([2.26295906e+07, 1.45683724e+08, 0.00000000e+00]), array([2.20474742e+07, 1.45778353e+08, 0.00000000e+00]), array([2.32113033e+07, 1.45586774e+08, 0.00000000e+00])])
Old B: deque([array([ 3.04180445e+07,  2.01114292e+08, -8.42648546e+06]), array([ 3.12208814e+07,  2.00992336e+08, -8.36592382e+06]), array([ 3.20236483e+07,  2.00869930e+08, -8.30534345e+06])])
Nu no shift 0.0
New B: deque([array([ 3.04180445e+07,  2.01114292e+08, -8.42648546e+06]

In [30]:
%timeit [x*pi/180 for x in o_earth[3:]]

1.42 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [32]:
print([x*pi/180 for x in o_earth[3:]])

[1.1595534132637177, 0.2576251885469097]


In [31]:
%timeit np.deg2rad(o_earth[3:])

4.32 µs ± 479 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
#inc, argpe, omega
rotate(np.deg2rad(-90),np.deg2rad(0),np.deg2rad(0),np.deg2rad(0),np.deg2rad(0),np.deg2rad(0))

[[ 1.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]]


In [32]:
print(arr[7:16])

[ 28 -92  11 -14 -86 -36 -54 -67   8]


In [54]:
%timeit minima(arr)

27.1 µs ± 986 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
%timeit minima(np.random.rand(400))

37.8 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
x_b = 0.5*( np.cos(10))
y_b = 0.5*( np.sin(10))
%timeit np.sqrt(x_b**2 + y_b**2)

2.53 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
%timeit sqrt(x_b**2 + y_b**2)

507 ns ± 55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [24]:
foe = np.array([1.243e2,2.5e3,3.234e2])
fum = np.array([7.22e2,6.234e3,9.124e3])
def dist(fum,foe):
    return sqrt((fum[0]-foe[0])**2 + (fum[1]-foe[1])**2 + (fum[2]-foe[2])**2)
%timeit np.sqrt(sum((fum-foe))**2)
# np.sqrt(np.sum((fum-foe)**2))
%timeit sqrt(sum((fum-foe))**2)
%timeit dist(fum,foe)
%timeit sqrt((fum[0]-foe[0])**2 + (fum[1]-foe[1])**2 + (fum[2]-foe[2])**2)

6.38 µs ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.73 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.65 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.42 µs ± 638 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [23]:
print( np.sqrt(sum((fum-foe)**2)))
# np.sqrt(np.sum((fum-foe)**2))
print( sqrt(sum((fum-foe)**2)))
print( sqrt((fum[0]-foe[0])**2 + (fum[1]-foe[1])**2 + (fum[2]-foe[2])**2))
%timeit np.linalg.norm(fum-foe)

9578.651348180494
9578.651348180494
9578.651348180494
5.43 µs ± 319 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
%timeit 0.5*( cos(barf[0]))

427 ns ± 172 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
barf = np.array([1,2,3])
%timeit 0.5*( np.cos(barf[0]))

1.92 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
print(type(x_b), type(y_b))

<class 'numpy.float64'> <class 'numpy.float64'>


In [5]:
%timeit np.linalg.norm([x_b,y_b])

5.81 µs ± 741 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
